In [11]:
import cv2
import numpy as np


def create_video_capture() -> dict[int, np.ndarray]:
    # define a video capture object
    vid = cv2.VideoCapture(0)
    video_dict = {}
    i = 0

    while (True):

        # Capture the video frame
        # by frame
        ret, frame = vid.read()
        video_dict[i] = frame
        i += 1
        # Display the resulting frame
        cv2.imshow('frame', frame)

        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()
    return video_dict


# https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html
def convert_to_gray_imgs(dict_imgs: dict[int, np.ndarray], k_size: tuple[int, int] = (3, 3)) -> None:
    for key, val in dict_imgs.items():
        print("key : {}".format(key))
        print("val shape : {}".format(val.shape))

In [12]:
dict_val = create_video_capture()
convert_to_gray_imgs(dict_val)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
key : 0
val shape : (480, 640, 3)
key : 1
val shape : (480, 640, 3)
key : 2
val shape : (480, 640, 3)
key : 3
val shape : (480, 640, 3)
key : 4
val shape : (480, 640, 3)
key : 5
val shape : (480, 640, 3)
key : 6
val shape : (480, 640, 3)
key : 7
val shape : (480, 640, 3)
key : 8
val shape : (480, 640, 3)
key : 9
val shape : (480, 640, 3)
key : 10
val shape : (480, 640, 3)
key : 11
val shape : (480, 640, 3)
key : 12
val shape : (480, 640, 3)
key : 13
val shape : (480, 640, 3)
key : 14
val shape : (480, 640, 3)
key : 15
val shape : (480, 640, 3)
key : 16
val shape : (480, 640, 3)
key : 17
val shape : (480, 640, 3)
key : 18
val shape : (480, 640, 3)
key : 19
val shape : (480, 640, 3)
key : 20
val shape : (480, 640, 3)
key : 21
val shape : (480, 640, 3)
key : 22
val shape : (480, 640, 3)
key : 23
val shape : (480, 640, 3)
key : 24
val shape :